<a href="https://colab.research.google.com/github/marcosverabower/1er-Final-IA/blob/main/RedNeuronalTempAsu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 9.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import optuna
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
tf.config.list_physical_devices('GPU')
tf.config.experimental.enable_op_determinism()
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.metrics import RootMeanSquaredError
SEED = 7

data=pd.read_csv(r'/content/Temp_Asu20092021.csv')

data['Fecha'] = pd.to_datetime(data['Fecha'])

data.set_index('Fecha', inplace=True)

data = data.drop(columns=data.columns[data.columns.str.strip() == 'Unnamed: 0'])
data.dropna()
data['Humedad'] = data['Humedad'].apply(lambda x: 100 if x > 100 else x)

#data = data[data.index.year >= 2019]
# Drop rows with missing values
data.dropna(inplace=True)
# Drop rows with missing values
data.dropna(inplace=True)
# Resample the dataset to daily intervals and calculate maximum temperature for each day
daily_max_temperatures = data['Temperatura'].resample('D').max()

# Apply quartile cut to the maximum daily temperatures
quartiles = daily_max_temperatures.quantile([0.25, 0.5, 0.75])

# Define temperature categories based on quartiles
cold_threshold = quartiles[0.25]
good_threshold = quartiles[0.5]
hot_threshold = quartiles[0.75]
print(quartiles)
data['Prom_Temperature_Day'] = data.groupby(data.index.date)['Temperatura'].transform('mean')

# Assign each temperature observation to its corresponding category
data['Temperature_Category'] = pd.cut(data['Prom_Temperature_Day'], bins=[-float('inf'), cold_threshold, good_threshold, hot_threshold, float('inf')],
                                      labels=['Cold', 'Good', 'Hot', 'Very Hot'])

# Shift the target variable by 24 rows to predict the category for the next day
data['NextDay_Temp'] = data['Temperature_Category'].shift(-8)


data_encoded = pd.get_dummies(data, columns=['Temperature_Category','NextDay_Temp'])


# Define features and target variable
X = data_encoded.iloc[:,0:7]
y = data_encoded.iloc[:,7:11]


# Normalize numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)



0.25    25.423505
0.50    30.311064
0.75    34.279108
Name: Temperatura, dtype: float64


In [ ]:
data

,Temperatura,Humedad,Prom_Temperature_Day,Temperature_Category,NextDay_Temp
Fecha,,,,,
2009-01-01 00:00:00,26.749483,69.609451,27.536757,Good,Cold
2009-01-01 03:00:00,25.956114,78.225166,27.536757,Good,Cold
2009-01-01 06:00:00,23.707357,93.551756,27.536757,Good,Cold
2009-01-01 09:00:00,23.726823,93.394817,27.536757,Good,Cold
2009-01-01 12:00:00,29.606379,72.155839,27.536757,Good,Cold
...,...,...,...,...,...
2021-12-31 09:00:00,21.719499,85.092387,31.041313,Hot,NaN
2021-12-31 12:00:00,29.548918,53.780205,31.041313,Hot,NaN
2021-12-31 15:00:00,40.026148,27.392646,31.041313,Hot,NaN


In [ ]:
print(X)

                     Temperatura    Humedad
Fecha                                      
2009-01-01 00:00:00    26.749483  69.609451
2009-01-01 03:00:00    25.956114  78.225166
2009-01-01 06:00:00    23.707357  93.551756
2009-01-01 09:00:00    23.726823  93.394817
2009-01-01 12:00:00    29.606379  72.155839
...                          ...        ...
2021-12-31 09:00:00    21.719499  85.092387
2021-12-31 12:00:00    29.548918  53.780205
2021-12-31 15:00:00    40.026148  27.392646
2021-12-31 18:00:00    41.244712  19.955186
2021-12-31 21:00:00    37.645491  22.674331

[37984 rows x 2 columns]


In [ ]:

# Split data into training and validation sets
train_indices = (data.index.year <= 2019)  #
val_indices = ((data.index.year >= 2020) & (data.index.year <= 2020))  #

# Split data into training and validation sets
X_train, X_val = X_scaled[train_indices], X_scaled[val_indices]
y_train, y_val = y[train_indices], y[val_indices]

print(X_train, X_val)
print(y_train, y_val)

[[ 4.87496194e-01  1.46693672e-03  7.43527803e-01 ...  1.41466046e+00
  -2.91563271e-01 -5.23976349e-02]
 [ 3.74328867e-01  4.07218555e-01  7.43527803e-01 ...  1.41466046e+00
  -2.91563271e-01 -5.23976349e-02]
 [ 5.35627779e-02  1.12901463e+00  7.43527803e-01 ...  1.41466046e+00
  -2.91563271e-01 -5.23976349e-02]
 ...
 [ 2.37494559e+00 -1.91438879e+00  1.27615868e+00 ... -7.06883402e-01
   3.42978729e+00 -5.23976349e-02]
 [ 1.85078523e+00 -1.68058475e+00  1.27615868e+00 ... -7.06883402e-01
   3.42978729e+00 -5.23976349e-02]
 [-1.45308821e-01  1.17754740e+00  1.27615868e+00 ... -7.06883402e-01
   3.42978729e+00 -5.23976349e-02]] [[-0.00252002  0.42812728  0.82305184 ...  1.41466046 -0.29156327
  -0.05239763]
 [-0.19605156  1.01951865  0.82305184 ...  1.41466046 -0.29156327
  -0.05239763]
 [-0.0057674   0.76218642  0.82305184 ...  1.41466046 -0.29156327
  -0.05239763]
 ...
 [ 0.25211628  0.57952327  0.56623742 ...  1.41466046 -0.29156327
  -0.05239763]
 [ 0.63945469  0.06432633  0.566237

In [ ]:
def objective(trial):
    # Define hyperparameters to be tuned
    num_layers = 1
    tf.keras.utils.set_random_seed(SEED)

    num_units = trial.suggest_categorical('num_units', [32, 64,128])#trial.suggest_int('num_units', 32, 128)
    learning_rate = trial.suggest_categorical('learning_rate', [1e-1, 1e-2,1e-3])#trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)

    # Define the model architecture
    model = Sequential()
    model.add(Dense(num_units, activation='relu', input_shape=(X.shape[1],)))
    for _ in range(num_layers - 1):
        model.add(Dense(num_units, activation='relu'))
    model.add(Dense(4, activation='softmax'))  # 4 categories: Cold, Good, Hot, Very Hot

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
 # Train the model
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=256, verbose=0,shuffle=False)

    # Evaluate the model on validation set
    _, val_acc = model.evaluate(X_val, y_val, verbose=0)

    return val_acc


In [ ]:
# Perform hyperparameter optimization using Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Get the best hyperparameters
best_num_layers = 1
best_num_units = study.best_params['num_units']
best_learning_rate = study.best_params['learning_rate']

print("Best Hyperparameters:")
print("Number of Layers:", best_num_layers)
print("Number of Units:", best_num_units)
print("Learning Rate:", best_learning_rate)


[I 2024-06-19 22:44:05,445] A new study created in memory with name: no-name-fa9e7af2-4f1f-4603-9ba7-fb8ab0bc544c
[I 2024-06-19 22:44:24,313] Trial 0 finished with value: 0.7329235076904297 and parameters: {'num_units': 128, 'learning_rate': 0.001}. Best is trial 0 with value: 0.7329235076904297.
[I 2024-06-19 22:44:44,774] Trial 1 finished with value: 0.7233606576919556 and parameters: {'num_units': 32, 'learning_rate': 0.1}. Best is trial 0 with value: 0.7329235076904297.
[I 2024-06-19 22:45:06,600] Trial 2 finished with value: 0.7339481115341187 and parameters: {'num_units': 128, 'learning_rate': 0.01}. Best is trial 2 with value: 0.7339481115341187.
[I 2024-06-19 22:45:27,428] Trial 3 finished with value: 0.7339481115341187 and parameters: {'num_units': 128, 'learning_rate': 0.01}. Best is trial 2 with value: 0.7339481115341187.
[I 2024-06-19 22:45:44,148] Trial 4 finished with value: 0.7233606576919556 and parameters: {'num_units': 32, 'learning_rate': 0.1}. Best is trial 2 with v

Best Hyperparameters:
Number of Layers: 1
Number of Units: 32
Learning Rate: 0.01


In [ ]:
final_model = Sequential()
final_model.add(Dense(best_num_units, activation='relu', input_shape=(X_train.shape[1],)))
final_model.add(Dense(best_num_units, activation='relu'))
final_model.add(Dense(4, activation='softmax'))

final_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=best_learning_rate),loss='categorical_crossentropy',
                  metrics=['accuracy'])

In [ ]:
final_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=12, verbose=1,shuffle=False)


Epoch 1/50
2678/2678 [==============================] - 10s 3ms/step - loss: 0.4909 - accuracy: 0.8029 - val_loss: 0.6641 - val_accuracy: 0.6800
Epoch 2/50
2678/2678 [==============================] - 8s 3ms/step - loss: 0.4552 - accuracy: 0.8070 - val_loss: 0.7178 - val_accuracy: 0.6393
Epoch 3/50
2678/2678 [==============================] - 9s 3ms/step - loss: 0.4551 - accuracy: 0.8084 - val_loss: 0.6466 - val_accuracy: 0.6612
Epoch 4/50
2678/2678 [==============================] - 7s 3ms/step - loss: 0.4377 - accuracy: 0.8171 - val_loss: 0.6331 - val_accuracy: 0.6885
Epoch 5/50
2678/2678 [==============================] - 9s 3ms/step - loss: 0.5405 - accuracy: 0.8157 - val_loss: 0.6500 - val_accuracy: 0.6711
Epoch 6/50
2678/2678 [==============================] - 7s 3ms/step - loss: 0.4354 - accuracy: 0.8153 - val_loss: 0.6331 - val_accuracy: 0.6742
Epoch 7/50
2678/2678 [==============================] - 9s 3ms/step - loss: 0.4348 - accuracy: 0.8173 - val_loss: 0.6144 - val_accuracy

In [ ]:
_, val_acc = final_model.evaluate(X_val, y_val, verbose=0)
print(val_acc)

0.6752049326896667
